In [72]:
import warnings
import pandas as pd

# Generating Prompts Using Structure and Synonyms

In this notebook, we demonstrate a protocol for generating search engine prompts of a given structure. We propose that prompts should be of the form "Watercraft" + "Crime" + "Consequence" + "Specifics" where each term in the sequence is randomly generated from a list of keywords. The "Specifics" term should be optional and can be used to generate prompts that can yield more specific incident reports. Examples of specific information that could be included are types of fish and geographic areas.

## Generating Keyword Lists for Each Term

Our first step is to generate lists of keywords for each term in our prompt. To this end, we use the database WordNet to find large lists of synonyms and related words for each term. WordNet is accessed in Python using the `nltk` library.

In [2]:
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/jackkendrick/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


We will use WordNet in the following way: First, we manually decide on the highest level of abstraction for each term that we should consider and then generate all synonyms of this word. In the case that a term has many meanings, each different usage is encoded as a separate entity within WordNet and so we must be careful to use the correct one. Once we have found all synonyms, we then find all 'hyponyms' (words with more specific meanings) of our term to form a keyword list.

First, we will generate a list of words for watercraft using WordNet. The highest level of abstraction we will consider here is "Craft". 

In [26]:
craft = wn.synsets('craft', 'n')[1]               # A synset is a collection of synonyms in WordNet - the function synsets generates a synset for each
hypos = lambda s:s.hyponyms()                       # meaning of the word. In our case, the relevant meaning of 'vessel' is given in the second synset

craft_hyponyms = list(craft.closure(hypos))              # The function hypos retrives all hyponyms from a synset. The function closure ensures we see all related terms.

for hypo in craft_hyponyms[:]:
    if wn.synset('fishing_boat.n.01').path_similarity(hypo) < 0.3:      # Here we are making sure that our keywords stay relevant. The similarity parameter can be changed.
        craft_hyponyms.remove(hypo)

craft_hyponyms

/Users/jackkendrick/anaconda3/envs/ncis/lib/python3.11/site-packages/nltk/corpus/reader/wordnet.py:604: UserWarning: Discarded redundant search for Synset('stealth_bomber.n.01') at depth 5
  for synset in acyclic_breadth_first(self, rel, depth):
/Users/jackkendrick/anaconda3/envs/ncis/lib/python3.11/site-packages/nltk/corpus/reader/wordnet.py:604: UserWarning: Discarded redundant search for Synset('stealth_fighter.n.01') at depth 5
  for synset in acyclic_breadth_first(self, rel, depth):


[Synset('vessel.n.02'),
 Synset('bareboat.n.01'),
 Synset('boat.n.01'),
 Synset('fishing_boat.n.01'),
 Synset('galley.n.01'),
 Synset('galley.n.02'),
 Synset('iceboat.n.02'),
 Synset('patrol_boat.n.01'),
 Synset('sailing_vessel.n.01'),
 Synset('ship.n.01'),
 Synset('shrimper.n.01'),
 Synset('weather_ship.n.01'),
 Synset('yacht.n.01'),
 Synset('trawler.n.02')]

The above code generates a list of relevant terms as a collection of 'synsets'. We can access each word in the set as follows:

In [101]:
craft_keywords = set()                        # We are using the set object to ensure that no word is added to our list twice.

for hypo in craft_hyponyms:
    for l in hypo.lemmas():
        name = l.name().replace('_', ' ')
        craft_keywords.add(name)

craft_keywords=list(craft_keywords)

We now repeat similar processes for consequences and other specific information. The following function can be used to automate this process:

In [102]:
def get_synonyms(concept:str, abst:int, control:str = None, control_param:float = None) -> list:
    '''
    Function for getting synonyms at varying levels of abstraction.

    concept : Concept as defined in WordNet. Should be of the form word.n.01.
    abst : Levels of abstraction. Function will generate synonyms by finding hypernyms at this level of abstraction.
    control : A base concept that generated synonyms should be similar to. If None, function will return all generated synonyms.
    control_param : Parameter for how similar synonyms should be to control. Should be a value between 0 and 1 smaller values allowing for less similarity.

    Returns list of generated synonyms as strings.
    '''

    warnings.filterwarnings('ignore') 
    
    syn = wn.synset(concept)
    for _ in range(abst):
        if len(syn.hypernyms()) >0:
            syn = syn.hypernyms()[0]

    hypos = lambda s:s.hyponyms()                       
    hyponyms = list(syn.closure(hypos))

    if control != None:   
        for hypo in hyponyms[:]:
            if wn.synset(control).path_similarity(hypo) < control_param:
                hyponyms.remove(hypo)

    words = set()                        

    for hypo in hyponyms:
        for l in hypo.lemmas():
            name = l.name().replace('_', ' ')
            words.add(name)

    return list(words)
    

One issue with this function is that it requires the user to specify as concept as it is stored in WordNet. It is possible that this can be fixed later.

We now use the above function to generate synonyms for consequences and some specifics that may be included in our prompts.

In [103]:
consequences = ['arrest.n.01', 'hearing.n.01', 'trial.n.04', 'fine.n.01']
abstractions = 2
similarity = 0.5

consequence_kw = []

for consequence in consequences:
    consequence_kw.extend(get_synonyms(consequence, abstractions, consequence, similarity))


In [104]:
for hypo in wn.synset('international_waters.n.01').closure(hypos):
    for l in hypo.lemmas():
        print(l.name())

In [105]:
specifics = ['seafood.n.01', 'water.n.02']
abstractions = 1
similarity = 0.5

specifics_kw = []

for specific in specifics:
    specifics_kw.extend(get_synonyms(specific, abstractions, specific, similarity))

We use a different method to generate the types of crimes that are in the category of underreporting/misreporting fishing catches. In the future, we hope that LLMs can be used to extract specific crimes from incident reports and these crimes can be used to create new prompts. In the mean time, we use a list of sample prompts and directly strip the crimes from them.

In [164]:
sample_prompts = pd.read_csv('sample-prompts.csv')
strip_vc = lambda prompt : prompt.split('Vessel caught ')[1]
crimes = list(set(sample_prompts['Prompt'].apply(strip_vc).values))

## Generating Prompts Using Keywords

Now for the fun part: Actually generating the prompts. We will generate a collection of prompts randomly using the keyword lists generated above. The following function will generate a specified number of prompts, with a user specified proportion being specific (i.e. includes either a geographic area or type of fish) with the remaining prompts being more general.

In [165]:
import numpy as np

In [166]:
def get_prompts(n:int, spec_prop:float) -> list:
    '''
    Generates a list of prompts with a proportion containing extra specific information.

    n : Number of prompts to be generated
    spec_prop : Proportion of prompts that should be specific. Float between 0 and 1

    Returns list of prompts as strings.
    '''

    prompts = []

    for _ in range(int(np.floor(n*(spec_prop)))):
        prompts.append(craft_keywords[np.random.randint(0, len(craft_keywords))] + ' ' + crimes[np.random.randint(0, len(crimes))] + ' ' 
                       + consequence_kw[np.random.randint(0, len(consequence_kw))] + ' ' + specifics_kw[np.random.randint(0, len(specifics_kw))])

    for _ in range(int(n-np.floor(n*(spec_prop)))):
        prompts.append(craft_keywords[np.random.randint(0, len(craft_keywords))] + ' ' + crimes[np.random.randint(0, len(crimes))] + ' ' 
                       + consequence_kw[np.random.randint(0, len(consequence_kw))])

    return prompts

In [167]:
prompts = get_prompts(100, 0.2)

## Searching the Web

We can now pass our prompts to a search engine and get links for the top results. To run this code, you must have a Google API key and a Custom Search Engine ID. Since these IDs should not be shared, I have my personal keys stored in a local file and import them below using some custom functions.

The code below was adapted from [this post on StackOverflow](https://stackoverflow.com/questions/37083058/programmatically-searching-google-in-python-using-custom-search) and you can find instructions for generating your own API and Custome Search Enginer keys there.

In [168]:
from googleapiclient.discovery import build
from googleapi import google_api_key, search_engine_id          # This is a local file that contains my personal API key and search engine ID.


my_api_key = google_api_key()
my_cse_id = search_engine_id()

The functions below define a custom search engine using the API key and Custom Search Engine ID and then returns the top 10 results as links.

In [175]:
def google_search(search_term, api_key, cse_id, **kwargs):
    service = build("customsearch", "v1", developerKey=api_key)
    res = service.cse().list(q=search_term, cx=cse_id, **kwargs).execute()

    if 'items' in res.keys():
        return res['items']

def get_links(prompt:str, api_key:str, cse_id:str, n=10):
    '''
    Takes in a search prompt and returns the top n results as links.

    prompt : Search query.
    api_key : Personal API key for accessing Google.
    cse_id : Custom Search Engine ID
    n : Number of results to return.

    Returns results as a list of urls.
    '''

    links =[]

    results = google_search(prompt, api_key, cse_id, num=n)
    if results != None:
        for result in results:
            links.append(result['link'])

    return links

We now iterate the process of searching and retrieving links over a long list of prompts. We store the links as a set so that no links are repeated.

In [178]:
links = set()

for prompt in list(sample_prompts['Prompt'].values):
    results = get_links(prompt, my_api_key, my_cse_id, 3)
    if results != None:
        for result in results:
            links.add(result)

In [179]:
links

{'https://news.google.com/__i/rss/rd/articles/CBMiK2h0dHBzOi8vd3d3LmhhemFyZHMub3JnL25ld3MvMjAxMC9pbmRleC5odG3SAQA?oc=5',
 'https://news.google.com/__i/rss/rd/articles/CBMiK2h0dHBzOi8vd3d3LnlvdXR1YmUuY29tL3dhdGNoP3Y9M043TDhYaGt6cW_SAQA?oc=5',
 'https://news.google.com/__i/rss/rd/articles/CBMiK2h0dHBzOi8vd3d3LnlvdXR1YmUuY29tL3dhdGNoP3Y9RTlFa085VXFoOUXSAQA?oc=5',
 'https://news.google.com/__i/rss/rd/articles/CBMiK2h0dHBzOi8vd3d3LnlvdXR1YmUuY29tL3dhdGNoP3Y9U3otNmpScmJ0dUnSAQA?oc=5',
 'https://news.google.com/__i/rss/rd/articles/CBMiK2h0dHBzOi8vd3d3LnlvdXR1YmUuY29tL3dhdGNoP3Y9Uk1oMVpLY0h1WUnSAQA?oc=5',
 'https://news.google.com/__i/rss/rd/articles/CBMiK2h0dHBzOi8vd3d3LnlvdXR1YmUuY29tL3dhdGNoP3Y9V0o5LXhONmRDVzTSAQA?oc=5',
 'https://news.google.com/__i/rss/rd/articles/CBMiK2h0dHBzOi8vd3d3LnlvdXR1YmUuY29tL3dhdGNoP3Y9Y0dhd1NPdXViQTDSAQA?oc=5',
 'https://news.google.com/__i/rss/rd/articles/CBMiK2h0dHBzOi8vd3d3LnlvdXR1YmUuY29tL3dhdGNoP3Y9Z2EzUmo5b2FNV0HSAQA?oc=5',
 'https://news.google.com/__i/rs

By inspection, the results here aren't what we would expect to see. ㅠㅡㅠ